# Etude des parser Beamer en Python

## Méthodologie

Nous allons étudier l'interêt de plusieurs parsers pour l'extraction d'information de présentations beamer :

TexSoup : https://github.com/alvinwan/TexSoup

LatexWalker : https://pylatexenc.readthedocs.io/en/latest/latexwalker/

PlasTeX : http://plastex.github.io/plastex/

LaTeXML : https://dlmf.nist.gov/LaTeXML/

Les deux premières sont des API qui permettent de modifier le LateX, les deux autres génèrent un document HTML ou XML, que l'on peut alors parser, avec par exemple https://lxml.de/

Le document beamer utilisé pour les tests est beamer/Beamer_test_parser.tex

### Objectifs des tests

Pour chaque parser, nous voulons tester leur capacité à :
- accèder à la structure hiérarchique
- accèder à la mise en forme du texte
- accèder aux métadonnées

La facilité des parsers à extraire ces données sera notre indicateur de leur pertinence pour notre projet.

### Tests à effectuer

Voici les tests à effectuer avec chaque parser : 

1) Donner : author, subject, date, keywords

2) Donner le nombre de diapositives

3) Donner la profondeur et le rang de la diapositive n°3

4) Donner le titre et le texte de la diapositive n°2, avec informations de mise en forme

Les attendus sont :

1) author : 'Samy Gascoin-Fontaine'

subject : 'test de parser beamer'

date : '20 novembre 2020'

keywords : 'test, Python, beamer, parser'

2) 4

3) profondeur : 3

rang : 1

4) titre : {text : 'première page', type : 'title'}

texte : {text : 'ceci est en italique', type : 'body', italic = True}


### Limites et difficultés

#### Le nombre de diapositives

Ce n'est pas immédiatement récupérable, notre méthode est de compter le nombre de '\begin{frame}' et d'ajouter 1 si l'on trouve '\maketitle' . Si d'autres commandes permettent d'ajouter des diapositives, il faut tester leur présence

#### La profondeur

Sur beamer, les sections autorisées sont \section, \subsection et \subsubsection. On a supposé ici pour simplifier que l'auteur utilise \section comme premier niveau de hiérarchie. On pourrait vérifier la présence de chaque, avant de leur attribuer une profondeur (dans le cas où l'utilisateur n'aurait que \subsection et \subsubsection)

#### Insuffisance de profondeur et rang

Le couple profondeur et rang de la diapositive ne suffit pas pour identifier une position unique dans une hiérarchie. Par exemple, une diapositive dans Section 1 sous-section 2 et une dans Section 2 sous-section 2 auront la même profondeur 2 (sous-section) et le même rang 2 (n°2 dans le niveau le plus profond, la sous-section). 

On pourrait imaginer le rang comme une liste, nous indiquant pour chaque niveau de la hiérarchie, son rang. Ici, les exemples donneraient [1,2] et [2,2]

## Evaluation des parsers

Actuellement, TexSoup semble de loin être le plus pertinent, car ses méthodes nous donnent facilement accès aux commandes, environnments, contenus et contient des fonctions de recherche. LatexWalker devrait permettre les mêmes opérations, mais la communauté est bien plus petite, et l'API est bien moins lisible.

PlasTeX et LaTeXML parsent le LateX et permettent une pseudo-équivalence XML. Ils sont donc moins pertinents, car ils nous faut encore parser le XML, avec un outil moins adapté que TexSoup l'est pour le LaTeX.

Nous n'avonspas pu faire aboutir les tests des parsers autres que TexSoup, mais leur intérêt pour notre tâche nous parait éminemment moindre.

## TexSoup version 0.3.1

Nous commençons avec TexSoup car il propose une API très lisible et apparemment suffisante pour parser le Latex. Il a également une communauté plus importante que LatexWalker, l'autre bibliothèque qui parse directement le Latex. 

### Ouvrir le fichier test beamer avec TexSoup

In [1]:
from TexSoup import TexSoup
with open('Beamer_test_parser.tex', 'r')as f:
    beamer_doc = f.read()
soup = TexSoup(beamer_doc)
soup

\documentclass{beamer}
\usepackage[french]{babel}
\usepackage[utf8]{inputenc}
\usepackage[T1]{fontenc}
\usetheme{Boadilla}

\title{presentation test n°1}
\author{Samy Gascoin-Fontaine}
\date{20 novembre 2020}
\subject{test de parser beamer}
%On suppose l'importation d'une commande keywords
\keywords{test, Python, beamer, parser}

\begin{document}

\maketitle
\section{Section 1}
\subsection{Sous-section 1}

\begin{frame}{première page}
\textit{ceci est en italique}
\end{frame}

\subsubsection{Sous-sous-section 1}

\begin{frame}{deuxième page}
\textbf{ceci est en gras}
\end{frame}

\subsection{Sous-section 2}

\begin{frame}{troisième page}
\underline{ceci est en souligné}
\end{frame}

\end{document}

### Test 1 : author, subject, date, keywords

In [2]:
author = soup.author.string
subject = soup.subject.string
date = soup.date.string
keywords = soup.keywords.string
for i in [author, subject, date, keywords] :
    print(i)

Samy Gascoin-Fontaine
test de parser beamer
20 novembre 2020
test, Python, beamer, parser


### Test 2 : le nombre de diapositives

In [3]:
nb_frame = 0
liste_begin_frame = soup.find_all(r'\begin{frame}')
nb_frame += len(liste_begin_frame)
if soup.maketitle != None :
    nb_frame += 1
print(nb_frame)

4


### Test 3 : profondeur et rang de la diapositive n°3

In [61]:
num_section, num_subsection, num_subsubsection, num_slide = 0,0,0,0
profondeur, rang = None, None
for child in soup.document.contents:
    if r'\maketitle' in str(child):
        num_slide += 1
    if r'\section' in str(child):
        num_section += 1
        num_subsection = 0
        num_subsubsection = 0
    if r'\subsection' in str(child):
        num_subsection += 1
        num_subsubsection = 0
    if r'\subsubsection' in str(child):
        num_subsubsection += 1
    if r'\begin{frame}' in str(child):
        num_slide += 1
        if num_slide == 3:
            if num_subsubsection > 0:
                profondeur = 3
                rang = num_subsubsection
            elif num_subsection > 0:
                profondeur = 2
                rang = num_subsection
            elif num_section > 0:
                profondeur = 1
                rang = num_section
if profondeur != None:
    print('la profondeur de la diapositive 3 est : ', profondeur)
else:
    print("la diapositive 3 n'est pas hiérarchisée")
if rang != None:
    print('le rang de la diapositive 3 est : ', rang)
else:
    print("la diapositive 3 n'est pas hiérarchisée")


la profondeur de la diapositive 3 est :  3
le rang de la diapositive 3 est :  1


### Test 4 : titre et contenu de la diapositive n°2, avec mise en forme

In [78]:
def mise_en_forme(content, typing):
    rv = {}
    if type(content.text) is list:
        rv['text'] = content.text[0]
    else :
        rv['text'] = content.text
    rv['type'] = typing
    if r'\textit' in str(content):
        rv['italic'] = True
    if r'\textbf' in str(content):
        rv['bold'] = True # etc.. pour chaque mise en forme
    return rv

num_slide = 0
for child in soup.document.contents:
    if r'\maketitle' in str(child):
        num_slide += 1
    if r'\begin{frame}' in str(child):
        num_slide += 1
        if num_slide == 2:
            title = child.contents[0]
            text = child.contents[1]

print('titre : ', mise_en_forme(title, 'title'))
print('texte : ', mise_en_forme(text, 'body'))

            

titre :  {'text': 'première page', 'type': 'title'}
texte :  {'text': 'ceci est en italique', 'type': 'body', 'italic': True}


## LatexWalker (pylatexenc version 2.8)

### Ouvrir le fichier test beamer avec LatexWalker

In [87]:
from pylatexenc.latexwalker import LatexWalker, LatexEnvironmentNode
with open('Beamer_test_parser.tex', 'r')as f:
    beamer = LatexWalker(f.read())
(nodelist, pos, len_) = beamer.get_latex_nodes(pos=0)
nodelist[0]

LatexMacroNode(parsing_state=<parsing state 140449885356384>, pos=0, len=22, macroname='documentclass', nodeargd=ParsedMacroArgs(argspec='[{', argnlist=[None, LatexGroupNode(parsing_state=<parsing state 140449885356384>, pos=14, len=8, nodelist=[LatexCharsNode(parsing_state=<parsing state 140449885356384>, pos=15, len=6, chars='beamer')], delimiters=('{', '}'))]), macro_post_space='')

L'API est assez obscure, je reviendrai plus tard

## PlasTeX version 2.1

L'orientation de PlasTeX est assez éloignée de ce que l'on veut faire, leur besoin est de transformer le LateX en HTML, et se concentrent sur le rendu graphique : http://plastex.github.io/plastex/tutorials/

## LaTeXML version 0.8.5

LaTeXML est uniquement une bibliothèque de conversion vers XML, et de ce fait, comme PlasTeX et